In [2]:
import pandas as pd
import numpy as np
from collections import Counter
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import tkinter as tk
from tkinter import ttk, messagebox


# Step 1: Load and clean datasets
train_df = pd.read_csv("C:\\Users\\SRINITHI K\\Desktop\\Crop Recommendation dataset\\Train Dataset.csv")
test_df = pd.read_csv("C:\\Users\\SRINITHI K\\Desktop\\Crop Recommendation dataset\\Test Dataset.csv")

# Drop unnamed columns
train_df = train_df.loc[:, ~train_df.columns.str.contains('^Unnamed')]
test_df = test_df.loc[:, ~test_df.columns.str.contains('^Unnamed')]

# Step 2: Encode target labels
label_encoder = LabelEncoder()
train_df['Crop'] = label_encoder.fit_transform(train_df['Crop'])
test_df['Crop'] = label_encoder.transform(test_df['Crop'])

# Step 3: Feature columns and data
feature_columns = train_df.columns.drop('Crop')
X_train = train_df[feature_columns].values
y_train = train_df['Crop'].values
X_test = test_df[feature_columns].values
y_test = test_df['Crop'].values

# ------------------------ Decision Tree Code ------------------------

def entropy(y):    #O(k)
    counts = np.bincount(y)
    probabilities = counts / len(y)
    return -np.sum([p * np.log2(p) for p in probabilities if p > 0])

def info_gain(y, left_indices, right_indices):   #O(k)
    parent_entropy = entropy(y)
    n = len(y)
    n_left = len(left_indices)
    n_right = len(right_indices)
    if n_left == 0 or n_right == 0:
        return 0
    left_entropy = entropy(y[left_indices])
    right_entropy = entropy(y[right_indices])
    child_entropy = (n_left/n) * left_entropy + (n_right/n) * right_entropy
    return parent_entropy - child_entropy

def best_split(X, y):   #O(n*d*k)
    best_gain = 0
    best_feature = None
    best_threshold = Non

    n_features = X.shape[1]
    for feature in range(n_features):
        thresholds = np.unique(X[:, feature])
        for t in thresholds:
            left = np.where(X[:, feature] <= t)[0]
            right = np.where(X[:, feature] > t)[0]
            gain = info_gain(y, left, right)
            if gain > best_gain:
                best_gain = gain
                best_feature = feature
                best_threshold = t

    return best_feature, best_threshold

class Node:
    def __init__(self, feature=None, threshold=None, left=None, right=None, value=None):
        self.feature = feature
        self.threshold = threshold
        self.left = left
        self.right = right
        self.value = value

def build_tree(X, y, depth=0, max_depth=10):   
    if len(set(y)) == 1 or depth == max_depth:
        most_common = Counter(y).most_common(1)[0][0]
        return Node(value=most_common)

    feature, threshold = best_split(X, y)
    if feature is None:
        return Node(value=Counter(y).most_common(1)[0][0])

    left_indices = np.where(X[:, feature] <= threshold)[0]
    right_indices = np.where(X[:, feature] > threshold)[0]

    left = build_tree(X[left_indices], y[left_indices], depth+1, max_depth)
    right = build_tree(X[right_indices], y[right_indices], depth+1, max_depth)

    return Node(feature=feature, threshold=threshold, left=left, right=right)

def predict_tree(x, tree):   #O(h)
    if tree.value is not None:
        return tree.value
    if x[tree.feature] <= tree.threshold:
        return predict_tree(x, tree.left)
    else:
        return predict_tree(x, tree.right)

# Build tree
tree = build_tree(X_train, y_train, max_depth=5)

# Predict on test set
y_pred = [predict_tree(x, tree) for x in X_test]
accuracy = round(accuracy_score(y_test, y_pred) * 100, 2)

# ------------------------ GUI Application ------------------------

class CropRecommendationApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Crop Recommendation System")
        self.root.geometry("900x650")
        self.root.configure(bg="#f0f0f0")
        
        # Create tabs
        self.tab_control = ttk.Notebook(root)
        
        self.prediction_tab = ttk.Frame(self.tab_control)
        self.about_tab = ttk.Frame(self.tab_control)
        
        self.tab_control.add(self.prediction_tab, text="Predict Crop")
        self.tab_control.add(self.about_tab, text="About")
        
        self.tab_control.pack(expand=1, fill="both")
        
        # Setup each tab
        self.setup_prediction_tab()
        self.setup_about_tab()
        
    def setup_prediction_tab(self):
        # Title
        title_frame = ttk.Frame(self.prediction_tab)
        title_frame.pack(pady=10)
        
        title_label = ttk.Label(title_frame, text="Crop Recommendation System", font=("Arial", 18, "bold"))
        title_label.pack()
        
        subtitle_label = ttk.Label(title_frame, text="Enter soil and climate parameters to predict the best crop", 
                                  font=("Arial", 12))
        subtitle_label.pack(pady=5)
        
        # Input fields
        input_frame = ttk.LabelFrame(self.prediction_tab, text="Input Parameters")
        input_frame.pack(pady=10, padx=20, fill="both", expand=True)
        
        # Dictionary to store entry widgets
        self.entries = {}
        
        # Create input fields dynamically
        for i, feature in enumerate(feature_columns):
            row = i // 2
            col = i % 2
            
            label_frame = ttk.Frame(input_frame)
            label_frame.grid(row=row, column=col, padx=20, pady=10, sticky="w")
            
            # Display friendly feature name
            feature_display = feature.replace("(", "").replace(")", "")
            label = ttk.Label(label_frame, text=f"{feature_display}:")
            label.grid(row=0, column=0, sticky="w", padx=5)
            
            # Entry widget with validation
            vcmd = (self.root.register(self.validate_number), '%P')
            entry = ttk.Entry(label_frame, width=15, validate="key", validatecommand=vcmd)
            entry.grid(row=0, column=1, sticky="w", padx=5)
            
            # No default values now
            self.entries[feature] = entry
        
        # Buttons
        button_frame = ttk.Frame(self.prediction_tab)
        button_frame.pack(pady=20)
        
        predict_button = ttk.Button(button_frame, text="Predict Crop", command=self.predict_crop)
        predict_button.grid(row=0, column=0, padx=10)
        
        clear_button = ttk.Button(button_frame, text="Clear Fields", command=self.clear_fields)
        clear_button.grid(row=0, column=1, padx=10)
        
        # Result frame
        self.result_frame = ttk.LabelFrame(self.prediction_tab, text="Prediction Result")
        self.result_frame.pack(pady=10, padx=20, fill="both", expand=True)
        
        self.result_label = ttk.Label(self.result_frame, text="Enter parameters and click 'Predict Crop'", 
                                     font=("Arial", 12), anchor="center")
        self.result_label.pack(pady=20, expand=True)
    
    
    
    def setup_about_tab(self):
        about_frame = ttk.Frame(self.about_tab)
        about_frame.pack(pady=20, padx=20, fill="both", expand=True)
        
        title = ttk.Label(about_frame, text="About This Project", font=("Arial", 16, "bold"))
        title.pack(pady=10)
        
        description = (
            "This application uses a Decision Tree algorithm to recommend the most suitable crop "
            "based on soil parameters (N, P, K, pH) and climate conditions (temperature, rainfall).\n\n"
            
            "The model was trained on agricultural data with various soil and climate parameters, "
            "achieving an accuracy of {}%.\n\n"
            
            "How to use:\n"
            "1. Enter the soil and climate parameters in the 'Predict Crop' tab\n"
            "2. Click the 'Predict Crop' button to get a recommendation\n"
            "3. View model performance details in the 'Model Results' tab"
        ).format(accuracy)
        
        desc_label = ttk.Label(about_frame, text=description, wraplength=600, justify="left")
        desc_label.pack(pady=10)
    
    def validate_number(self, value):
        # Allow empty field or valid number
        if value == "":
            return True
        try:
            float(value)
            return True
        except ValueError:
            return False
    
    def predict_crop(self):
        try:
            # Get input values
            input_values = []
            for feature in feature_columns:
                value = self.entries[feature].get()
                if not value:
                    messagebox.showerror("Input Error", f"Please enter a value for {feature}")
                    return
                value_type = float if "Temperature" in feature or "Rainfall" in feature or "pH" in feature else int
                input_values.append(value_type(value))
            
            # Make prediction
            input_array = np.array([input_values])
            prediction = predict_tree(input_array[0], tree)
            crop_name = label_encoder.inverse_transform([prediction])[0]
            
            # Update result label
            self.result_label.config(text=f"Recommended Crop: {crop_name}", font=("Arial", 14, "bold"))
            
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {str(e)}")
    
    def clear_fields(self):
        for entry in self.entries.values():
            entry.delete(0, tk.END)
        self.result_label.config(text="Enter parameters and click 'Predict Crop'")

# Run the application
if __name__ == "__main__":
    root = tk.Tk()
    app = CropRecommendationApp(root)
    root.mainloop()